In [ ]:
import numpy as np
import time
import pickle

from scipy.spatial.distance import pdist
from scipy.stats import linregress
from scipy.interpolate import interp1d
from sklearn.metrics import pairwise_distances_chunked
#from sklearn.decomposition import PCA
#from sklearn.manifold import Isomap
# from sklearn.manifold import LocallyLinearEmbedding
#from sklearn.manifold import SpectralEmbedding

from scipy.io import savemat

import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Load solution

In [ ]:
save_dir = "/home/users/axelwang/pyCycle/results/"
file_name = "n200dlP05dcP01whstar5P9.pickle"

with open(save_dir+file_name, 'rb') as handle:
    sol = pickle.load(handle)

In [ ]:
!ls /home/users/axelwang/pyCycle/results/

### Save solutions to .mat if needed

In [ ]:
mdict = {}
mdict['t'] = sol.t
mdict['y'] = sol.y
savemat(save_dir+'n200dlP05dcP01whstar5P9.mat',mdict)

## Input parameters

In [ ]:
### Domain
# number of elements on fault
n = 200
# element length (km)
dl = 0.05
# Fault length (km)
FL = n*dl
# Central VW length (km)
VWL = 6 
# shear wave velocity (km/s)
vs = 3 
# shear modulus (MPa)
mu=30
# radiation damping coefficient (MPa*s/m)
eta = mu/2/vs
# Normal stress (MPa)
sigma = 100
# Poisson ratio
pois = 0.25

### Rate-state
# Reference velocity (km/s)
vref = 1e-6
# reference friciton coefficient
fref = 0.6
# a
amin = 0.015   # for VW zone
amax = 0.025   # for VS zone
vspts = int(np.ceil((FL-VWL)/2/dl))   # number of elements in each side of VS zone
vwpts = int(np.ceil(VWL)/dl)          # number of elements in central VW zone
a = np.zeros(n)
a[0:vspts] = amax
a[vspts:vspts+vwpts] = amin
a[vspts+vwpts:] = amax
# b
b = 0.020
# state evolution distance (m)
dc = 0.01
# (Plate) loading velocity
vl = 1e-9

### Simulation time (s)
tmax = 1e10

print(FL)

## Plot

In [ ]:
# Pick out psi and f
psi_sol = np.zeros([n,len(sol.t)])
f_sol = np.zeros([n,len(sol.t)])
for i in range(n):
    psi_sol[i] = sol.y[2*i,:]
    f_sol[i] = sol.y[2*i+1,:]

# Build a geometry for fault
faultx = np.arange(0,n,1)*dl   # km
# Time steps
tsteps = np.arange(len(sol.t))

## Make meshgrids
X, Y = np.meshgrid(faultx,tsteps)

In [ ]:
# Calculate slip velocities
v_sol = np.zeros([n,len(sol.t)])
for i in range(n):
    v_sol[i,:] = 2*vref*np.sinh(f_sol[i,:]/a[i])*np.exp(-psi_sol[i,:]/a[i])

#### Slip velocity

In [ ]:
# fig,ax = plt.subplots(1,1,figsize=(12,10))
vplot = np.log10(v_sol.transpose())
# c = ax.pcolormesh(X,Y,vplot,cmap='viridis')
# plt.xlabel('Fault x (km)', fontsize=18)
# plt.ylabel('Time step #', fontsize=18)

In [ ]:
# get rid of the initial transients
cutStep= 200     # n =200
#cutStep = 6000  # n= 500
#cutStep = 8000   # n= 1000


X_cut = X[cutStep:-1,:]
Y_cut = Y[cutStep:-1,:]
vplot_cut = vplot[cutStep:-1,:]
fplot_cut = f_sol.transpose()[cutStep:-1,:]
plt.rcParams.update({'font.size': 36})
fig,ax = plt.subplots(1,1,figsize=(12,10))
c = ax.pcolormesh(X_cut,Y_cut,vplot_cut,cmap='viridis')
#cb = fig.colorbar(c, ax=ax,location='top')
#cb.set_label(r'$log_{10}$ slip velocity (m/s)')
plt.xlabel('Fault x (km)')
plt.ylabel('Time step #')

#### Plot individual time series

In [ ]:
find_x = 3
fault_idx = (np.abs(faultx-find_x)).argmin()
time_cut = sol.t[cutStep:-1]
v_cut = vplot_cut[:,fault_idx]
f_cut = fplot_cut[:,fault_idx]
print(fault_idx)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_cut/(np.pi*1e7),
                         y=v_cut,
                         mode='markers',
                         marker=dict(color='red')))
fig.add_trace(go.Scatter(x=time_cut/(np.pi*1e7),
                         y=v_cut,
                         mode='lines',
                         marker=dict(color='blue')))

In [ ]:
## phase space of this single point f-v
fig = go.Figure()
fig.add_trace(go.Scatter(x=v_cut,
                         y=f_cut,
                         mode='lines',
                         marker=dict(color='black')))
fig.update_layout(width=800,height=800)



In [ ]:
find_x = 7.02
fault_idx = (np.abs(faultx-find_x)).argmin()
v_cut2 = vplot_cut[:,fault_idx]
f_cut2 = fplot_cut[:,fault_idx]
print(fault_idx)

In [ ]:
find_x = 3
fault_idx = (np.abs(faultx-find_x)).argmin()
v_cut3 = vplot_cut[:,fault_idx]
f_cut3 = fplot_cut[:,fault_idx]
print(fault_idx)

In [ ]:
## Data for dimensional reduction
# 2n-dimensional dataset
data = np.concatenate((vplot_cut,fplot_cut,),axis=1)

In [ ]:
## Time-delay embedding (Taken's theorem)
def takensEmbedding (data, delay, dimension):
    "This function returns the Takens embedding of data with delay into dimension, delay*dimension must be < len(data)"
    if delay*dimension > len(data):
        raise NameError('Delay times dimension exceed length of data!')    
    embeddedData = np.array([data[0:len(data)-delay*dimension]])
    for i in range(1, dimension):
        embeddedData = np.append(embeddedData, [data[i*delay:len(data) - delay*(dimension - i)]], axis=0)

    return embeddedData

In [ ]:
eb = takensEmbedding(data[:,fault_idx],50,3)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=eb[0,:],
                         y=eb[1,:],
                         z = eb[2,:],
                         mode='lines',
                         marker=dict(color='red')))
fig.update_layout(width=800,height=800,font=dict(size=16), title="n=200, Delay = 50")

fig.update_layout(scene = dict(
                    xaxis_title='X',
                    yaxis_title='Y',
                    zaxis_title='Z'),)

#### Friction coefficient 

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,10))
c = ax.pcolormesh(X_cut,Y_cut,fplot_cut,cmap='viridis')
plt.xlabel('Fault x (km)', fontsize=18)
plt.ylabel('Time step #', fontsize=18)

#### Shear stress

In [ ]:
tau_sol = sigma * f_sol

fig,ax = plt.subplots(1,1,figsize=(12,10))
c = ax.pcolormesh(X,Y,tau_sol.transpose(),cmap='viridis')
plt.xlabel('Fault x (km)', fontsize=18)
plt.ylabel('Time step #', fontsize=18)


## Correlation dimension

### Pairwise distance chunked

In [ ]:
allStates = sol.y.T
allStates = allStates[cutStep:-1,:]

####  Get the range of distances

In [ ]:
# chunked pairwise distance generator
chunked_distances = pairwise_distances_chunked(allStates,metric = 'euclidean')

# get the range of distances so we can set up distance ranges to count pairs
chunked_logmax = []
chunked_logmin = []
for c in chunked_distances:
    chunked_logmax.append(np.log10(np.max(c[c != 0])))
    chunked_logmin.append(np.log10(np.min(c[c != 0])))

rmin = np.min(chunked_logmin)
print(rmin)
rmax = np.max(chunked_logmax)
print(rmax)

In [ ]:
#neighborhood radius
r_full = np.logspace(-3, 1.5, 50)

#### Count the number of pairs within each distance range

In [ ]:
# generate again
chunked_distances = pairwise_distances_chunked(allStates,metric = 'euclidean')

# count the number of pairs within each distance range
chunked_counts = np.zeros(len(r_full))


st = time.time()
chunk_num = 0
total_num = len(chunked_logmin)
#total_num = 9

# search range
sr = np.log10(r_full)
for c in chunked_distances:

    # Get only the upper triangular matrix without the diagonal (this sets the rest to 0)
    c = np.triu(c,1)   
    # Delete zeros:
    c = c[c != 0] 
    # Take log10
    c = np.log10(c)
    
    # increment chunk 
    chunk_num +=1
    for i,r in enumerate(sr):
        chunked_counts[i] += np.count_nonzero(c <= r)
    
    print("finished chunk "+ str(chunk_num) + " of " + str(total_num))
    print("Time used: " +str(time.time()-st) + " s")
    print(" ")

et = time.time()

#### Calculate correlation integral

In [ ]:
N = len(allStates)
C_full = chunked_counts/N**2

In [ ]:
C_full

#### Plot

In [ ]:
ax = plt.figure().add_subplot()
ax.plot(np.log10(r_full),np.log10(C_full),'bo')

In [ ]:
fitstart = 3
fitend = -5
r_fit = r_full[fitstart:fitend]
C_fit = C_full[fitstart:fitend]
line_fit=linregress(np.log10(r_fit),np.log10(C_fit))
print(line_fit)

In [ ]:
plt.rcParams.update({'font.size': 36})
fig,ax = plt.subplots(1,1,figsize=(8,6.6))
ax.plot(np.log10(r_full),np.log10(C_full),'bo',markersize=8)
ax.plot(np.log10(r_fit),np.log10(C_fit),'r*',markersize=8)
ax.plot(np.log10(r_fit),line_fit.slope*np.log10(r_fit)+line_fit.intercept,color='black')
ax.set(xlabel=r'$log_{10} \: r $',ylabel=r'$log_{10} \: C(r) $')

#### pdist

In [ ]:
def correlation_integral(dist,r_range, N):
    '''
    dist: euclidean norm, without log being taken
    r_range: generated in logspace, but without log being taken
    N: number of total points/states of the system
    '''
     
    dist = np.log10(dist)
    r_range = np.log10(r_range)
    
    Cr = []
    for r in r_range:
        Cr.append(np.count_nonzero(dist < r)/N**2)
    
    return np.array(Cr)

In [ ]:
allStates = sol.y.T
allStates = allStates[cutStep:-1,:]

distances = pdist(allStates)

N = len(allStates)

In [ ]:
rmin = np.log10(np.min(distances))
print(rmin)

In [ ]:
rmax = np.log10(np.max(distances))
print(rmax)

In [ ]:
#neighborhood radius
r_full = np.logspace(-3, 1.2, 50)
C_full = correlation_integral(distances,r_full,N)

ax = plt.figure().add_subplot()
ax.plot(np.log10(r_full),np.log10(C_full),'bo')

In [ ]:
C_full*N**2

In [ ]:
fitstart = 9
fitend = -4
r_fit = r_full[fitstart:fitend]
C_fit = C_full[fitstart:fitend]
line_fit=linregress(np.log10(r_fit),np.log10(C_fit))
print(line_fit)

In [ ]:
ax = plt.figure().add_subplot()
ax.plot(np.log10(r_full),np.log10(C_full),'bo')
ax.plot(np.log10(r_fit),np.log10(C_fit),'r*')
ax.plot(np.log10(r_fit),line_fit.slope*np.log10(r_fit)+line_fit.intercept,color='black')